In [1]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

import fnmatch

# Set seed for experiment reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

### Ссылки на датасеты:
- Датасет с флейтами: https://zenodo.org/record/1408985#.YkXlzupBxPZ
- Датасет "CSD": https://zenodo.org/record/4785016#.YkYd-OpBxPZ

## Разбиение датасета с флейтами и датасета "CSD"

## Распределение нот по частотам
- E	Ми пятой октавы	5274.00
- E	Ми четвёртой октавы	2637.00
- E	Ми третьей октавы	1318.50
- E	Ми второй октавы	659.26
- E	Ми первой октавы	329.63
- E	Ми малой октавы	164.81
- E	Ми большой октавы	82.41
- E	Ми контроктавы	41.21
- E	Ми субконтроктавы	20.61
- 
- 
- D#	Ре-диез пятой октавы	4978.00
- D#	Ре-диез четвёртой октавы	2489.00
- D#	Ре-диез третьей октавы	1244.50
- D#	Ре-диез второй октавы	622.26
- D#	Ре-диез первой октавы	311.13
- D#	Ре-диез малой октавы	155.56
- D#	Ре-диез большой октавы	77.78
- D#	Ре-диез контроктавы	38.88
- 
- 
- D	Ре пятой октавы	4698.40
- D	Ре четвёртой октавы	2349.20
- D	Ре третьей октавы	1174.60
- D	Ре второй октавы	587.32
- D	Ре первой октавы	293.66
- D	Ре малой октавы	147.83
- D	Ре большой октавы	73.91
- D	Ре контроктавы	36.95
- 
- 
- C#	До-диез пятой октавы	4434.80
- C#	До-диез четвёртой октавы	2217.40
- C#	До-диез третьей октавы	1108.70
- C#	До-диез второй октавы	554.36
- C#	До-диез первой октавы	277.18
- C#	До-диез малой октавы	138.59
- C#	До-диез большой октавы	69.30
- C#	До-диез контроктавы	34.65
- 
- 
- C	До пятой октавы	4186.00
- C	До четвёртой октавы	2093.00
- C	До третьей октавы	1046.50
- C	До второй октавы	523.25
- C	До первой октавы	261.63
- C	До малой октавы	130.82
- C	До большой октавы	65.41
- C	До контроктавы	32.70
- 
- 
- B	Си четвёртой октавы	3951.00
- B	Си третьей октавы	1975.50
- B	Си второй октавы	987.75
- B	Си первой октавы	493.88
- B	Си малой октавы	246.96
- B	Си большой октавы	123.48
- B	Си контроктавы	61.74
- B	Си субконтроктавы	30.87
- 
- 
- A#	Ля-диез четвёртой октавы	3729.20
- A#	Ля-диез третьей октавы	1864.60
- A#	Ля-диез второй октавы	932.32
- A#	Ля-диез первой октавы	466.16
- A#	Ля-диез малой октавы	233.08
- A#	Ля-диез большой октавы	116.54
- A#	Ля-диез контроктавы	58.26
- A#	Ля-диез субконтроктавы	29.13
- 
- 
- A	Ля четвёртой октавы	3440.00
- A	Ля третьей октавы	1720.00
- A	Ля второй октавы	880.00
- A	Ля первой октавы	440.00
- A	Ля малой октавы	220.00
- A	Ля большой октавы	110.00
- A	Ля контроктавы	55.00
- A	Ля субконтроктавы	27.50
- 
- 
- G#	Соль-диез четвёртой октавы	3332.40
- G#	Соль-диез третьей октавы	1661.20
- G#	Соль-диез второй октавы	830.60
- G#	Соль-диез первой октавы	415.30
- G#	Соль-диез малой октавы	207.00
- G#	Соль-диез большой октавы	103.80
- G#	Соль-диез контроктавы	51.90
- G#	Соль-диез субконтроктавы	25.95
- 
- 
- G	Соль четвёртой октавы	3136.00
- G	Соль третьей октавы	1568.00
- G	Соль второй октавы	784.00
- G	Соль первой октавы	392.00
- G	Соль малой октавы	196.00
- G	Соль большой октавы	98.00
- G	Соль контроктавы	49.00
- G	Соль субконтроктавы	24.50
- 
- 
- F#	Фа-диез четвёртой октавы	2960.00
- F#	Фа-диез третьей октавы	1480.00
- F#	Фа-диез второй октавы	739.98
- F#	Фа-диез первой октавы	369.99
- F#	Фа-диез малой октавы	185.00
- F#	Фа-диез большой октавы	92.50
- F#	Фа-диез контроктавы	46.25
- F#	Фа-диез субконтроктавы	23.12
- 
- 
- F	Фа четвёртой октавы	2793.80
- F	Фа третьей октавы	1396.90
- F	Фа второй октавы	698.46
- F	Фа первой октавы	349.23
- F	Фа малой октавы	174.62
- F	Фа большой октавы	87.31
- F	Фа контроктавы	43.65
- F	Фа субконтроктавы	21.82


In [2]:
data_dir_audio1 = pathlib.Path(r"D:\magistracy\diplom\flute-audio-labelled-database-AMT\Recordings")

In [3]:
data_dir_audio2 = pathlib.Path(r"D:\magistracy\diplom\CSD\english\wav")

In [4]:
data_dir_audio3 = pathlib.Path(r"D:\magistracy\diplom\CSD\korean\wav")

In [5]:
filenames_audio1 = []
filenames_audio2 = []

for file in os.listdir(data_dir_audio1):
    if fnmatch.fnmatch(file,'*leg*.wav'):
        filenames_audio1.append(os.path.join(data_dir_audio1, file))
for file in os.listdir(data_dir_audio2):
    if fnmatch.fnmatch(file,'*.wav'):
        filenames_audio2.append(os.path.join(data_dir_audio2, file))
for file in os.listdir(data_dir_audio3):
    if fnmatch.fnmatch(file,'*.wav'):
        filenames_audio2.append(os.path.join(data_dir_audio3, file))
filenames_audio1[0:3]

['D:\\magistracy\\diplom\\flute-audio-labelled-database-AMT\\Recordings\\arpeg-AbMaj-060BPM-leg.wav',
 'D:\\magistracy\\diplom\\flute-audio-labelled-database-AMT\\Recordings\\arpeg-AbMaj-090BPM-leg.wav',
 'D:\\magistracy\\diplom\\flute-audio-labelled-database-AMT\\Recordings\\arpeg-AbMaj-120BPM-leg.wav']

In [6]:
filenames_audio2[0:3]

['D:\\magistracy\\diplom\\CSD\\english\\wav\\en001a.wav',
 'D:\\magistracy\\diplom\\CSD\\english\\wav\\en001b.wav',
 'D:\\magistracy\\diplom\\CSD\\english\\wav\\en002a.wav']

In [7]:
num_samples = len(filenames_audio1)+len(filenames_audio2)
print('Number of total examples:', num_samples)
print('Example file tensor:', filenames_audio1[0])

Number of total examples: 241
Example file tensor: D:\magistracy\diplom\flute-audio-labelled-database-AMT\Recordings\arpeg-AbMaj-060BPM-leg.wav


In [8]:
data_dir_txt = pathlib.Path(r"D:\magistracy\diplom\flute-audio-labelled-database-AMT\Aligned_files\text_files")

In [9]:
data_dir_csv1 = pathlib.Path(r"D:\magistracy\diplom\CSD\english\csv")

In [10]:
data_dir_csv2 = pathlib.Path(r"D:\magistracy\diplom\CSD\korean\csv")

In [11]:
filenames_txt = []

for file in os.listdir(data_dir_txt):
    #if file.endswith(r".txt") :
    if fnmatch.fnmatch(file,'*leg*.txt'):
        filenames_txt.append(os.path.join(data_dir_txt, file))
filenames_txt[0:3]

['D:\\magistracy\\diplom\\flute-audio-labelled-database-AMT\\Aligned_files\\text_files\\arpeg-AbMaj-060BPM-leg-lined.txt',
 'D:\\magistracy\\diplom\\flute-audio-labelled-database-AMT\\Aligned_files\\text_files\\arpeg-AbMaj-090BPM-leg-lined.txt',
 'D:\\magistracy\\diplom\\flute-audio-labelled-database-AMT\\Aligned_files\\text_files\\arpeg-AbMaj-120BPM-leg-lined.txt']

In [12]:
filenames_csv = []

for file in os.listdir(data_dir_csv1):
    #if file.endswith(r".txt") :
    if fnmatch.fnmatch(file,'*.csv'):
        filenames_csv.append(os.path.join(data_dir_csv1, file))
for file in os.listdir(data_dir_csv2):
    #if file.endswith(r".txt") :
    if fnmatch.fnmatch(file,'*.csv'):
        filenames_csv.append(os.path.join(data_dir_csv2, file))
filenames_csv[0:3]

['D:\\magistracy\\diplom\\CSD\\english\\csv\\en001a.csv',
 'D:\\magistracy\\diplom\\CSD\\english\\csv\\en001b.csv',
 'D:\\magistracy\\diplom\\CSD\\english\\csv\\en002a.csv']

In [13]:
num_txt_csv = len(filenames_txt)+len(filenames_csv)
print('Number of total examples:', num_txt_csv)
print('Example file tensor (txt):', filenames_txt[0])
print('Example file tensor (csv):', filenames_csv[0])

Number of total examples: 241
Example file tensor (txt): D:\magistracy\diplom\flute-audio-labelled-database-AMT\Aligned_files\text_files\arpeg-AbMaj-060BPM-leg-lined.txt
Example file tensor (csv): D:\magistracy\diplom\CSD\english\csv\en001a.csv


Математическая формула частоты для всего звукоряда имеет следующий вид:  
$$
  f(i)=f_0\bullet2^{i/12},\quad\quad\quad(1)
$$  
где $f_0$ - частота камертона (обычно соответствует ноте A и равна 440 Гц), $i$ - количество полутонов в интервале от исследуемого звука к эталону $f_0$.  
Для нахождения количества полутонов, на которое заданная частота $f(i)$ отстает от частоты камертона $f_0$, используется формула, полученная из (1):  
$$
i = \frac{\ln{\frac{f_0}{f(i)}}}{\ln{2^{\frac{1}{12}}}}.\quad\quad\quad(2)
$$  

Для сведения всех нот к 12 основным, необходимо найти $z = round(i) \% 12 = y \% 12$. При этом, если $z<0$, то разворачиваем ноту так, чтобы она оказалась выше ноты A. Отсюда получаем:
$$
z=
\begin{cases}
   y \% 12, &\text{ $z\geq0$},\\
   12 + y \% 12, &\text{ $z<0$}.
 \end{cases}
\quad\quad\quad (3)
$$  
Нота, соответствующая заданной частоте без учета октав, определяется из значения $z$ по таблице:

| z | Нота | z | Нота | z | Нота | z | Нота |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 0 | A | 3 | F# | 6 | D# | 9 | C |
| 1 | G# | 4 | F | 7 | D | 10 | B |
| 2 | G | 5 | E | 8 | C# | 11 | A# |

Таким образом, зная разбиение мелодий по времени на ноты и соответствующие этим нотам частоты, можно разделить аудио на записи конкретных нот, распределив их по директориям, соответствующим названиям нот.

In [14]:
def freq_distrib(freq):
    note = None
    
    y = np.log(440/freq)/np.log(2**(1/12))
    z = round(y)%12
    if z < 0:
        z += 12 
       
    n = {0: 'A/',
         1: 'G#/',
         2: 'G/',
         3: 'F#/',
         4: 'F/',
         5: 'E/',
         6: 'D#/',
         7: 'D/',
         8: 'C#/',
         9: 'C/',
         10: 'B/',
         11: 'A#/'}
    note = n[z]

    return note

In [15]:
import shutil

try:
    shutil.rmtree("records_cut")
except OSError as e:
    pass

os.makedirs("records_cut")
os.makedirs("records_cut/A")
os.makedirs("records_cut/G#")
os.makedirs("records_cut/G")
os.makedirs("records_cut/F#")
os.makedirs("records_cut/F")
os.makedirs("records_cut/E")
os.makedirs("records_cut/D#")
os.makedirs("records_cut/D")
os.makedirs("records_cut/C#")
os.makedirs("records_cut/C")
os.makedirs("records_cut/B")
os.makedirs("records_cut/A#")

В датасете "CSD" вместо частот заданы высоты тонов (pitches). Для перехода к частотам используем стандарт midi:  
$$f = 440\bullet2^{\frac{p-69}{12}},\quad\quad\quad (4)$$ 
где $f$ - частота в Гц, $p$ - высота тона.  
  
  Для удобства переведем секунды в отсчеты. Частота мелодий в датасете равна 44100 Гц, т.е. 44100 отсчетов в секунду. Значит, для перевода времени в отсчеты воспользуемся формулой (5):
  $$n = round(t)\bullet44100,\quad\quad\quad (5)$$
где $n$ - количество отсчетов, $t$ - время в секундах.

In [16]:
from scipy.io import wavfile
import scipy.io
import csv

for f_num in range(len(filenames_txt)):
    f = open(filenames_txt[f_num], 'r')
    l = [line.strip() for line in f]
    l = l[4:len(l)-1]
    time_start = []
    time_end = []
    frequency = []

    for a in l:
        start,length,freq = a.split(' ')
        time_start.append(int(np.floor(float(start)*44100)))
        time_end.append(int(np.ceil((float(start)+float(length))*44100)))
        frequency.append(float(freq))
    f.close()

    _ , data = wavfile.read(filenames_audio1[f_num])
    waveform = data
    
    for k in range(len(time_start)):
        
        note = freq_distrib(frequency[k])
        
        if note!=None:
            file_name = 'records_cut/'+note+filenames_audio1[f_num][len(str(data_dir_audio1))+1:len(filenames_audio1[f_num])-4]+'_'+str(k+1)
            scipy.io.wavfile.write(file_name+'.wav', 44100, waveform[time_start[k]:time_end[k]+1])

for f_num in range(len(filenames_csv)):
    #if f_num not in [58,59]:
    time_start = []
    time_end = []
    frequency = []
    pitch = []
    with open(filenames_csv[f_num]) as f:
        reader = csv.reader(f)
        for row in reader:
            time_start.append(row[0])
            time_end.append(row[1])
            frequency.append(row[2])
            pitch.append(row[2])
    time_start = list(map(float,time_start[1:len(time_start)+1]))
    time_end = list(map(float,time_end[1:len(time_end)+1]))
    frequency = list(map(float,frequency[1:len(frequency)+1]))
    pitch = list(map(float,pitch[1:len(pitch)+1]))
    for i in range(len(time_start)):
        time_start[i] = int(np.floor(time_start[i]*44100))
        time_end[i] = int(np.floor(time_end[i]*44100))
        frequency[i] = 440*2**((frequency[i]-69)/12) # Перевод pitch в frequency

    _ , data = wavfile.read(filenames_audio2[f_num])

    if f_num not in [58,59]:
        waveform = data[:,0]
    else:
        waveform = data


    for k in range(len(time_start)):

        note = freq_distrib(frequency[k])

        if note!=None:
            file_name = 'records_cut/'+note+filenames_audio2[f_num][len(filenames_audio2[f_num])-10:len(filenames_audio2[f_num])-4]+'_'+str(k+1)
            scipy.io.wavfile.write(file_name+'.wav', 44100, waveform[time_start[k]:time_end[k]+1])


C:\Users\belas\AppData\Local\Temp/ipykernel_9640/1727488073.py:53: WavFileWarning: Chunk (non-data) not understood, skipping it.
  _ , data = wavfile.read(filenames_audio2[f_num])
